# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [36]:
finetuning = 1
inference = not finetuning

# Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Data

In [6]:
data = pd.read_csv('/content/drive/MyDrive/#Research/# GB/v2(gender_code_change)_updated_senti.csv')
data

,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked
0,সাইন্সের মেয়েরা আসলেই একটু অহংকারী ও দাম্বিক হ...,সাইন্সের ছেলেরা আসলেই একটু অহংকারী ও দাম্বিক হ...,['লামিয়া'],0,0,0,0,সাইন্সের <Gender> আসলেই একটু অহংকারী ও দাম্বিক...,সাইন্সের <Gender> আসলেই একটু অহংকারী ও দাম্বিক...
1,মেহজাবিন এর ছাত্রী এটা? এই ছাগীটাকে প্রায় নাটক...,মেহজাবিন এর ছাত্র এটা? এই ছাগটাকে প্রায় নাটকেই...,['মেহজাবিন'],0,0,0,0,মেহজাবিন এর <Gender> এটা? এই ছাগীটাকে প্রায় না...,<Name> এর <Gender> এটা? এই ছাগীটাকে প্রায় নাটক...
2,মাইয়া টা খালি কান্দে,পোলা টা খালি কান্দে,[],0,0,0,0,<Gender> টা খালি কান্দে,<Gender> টা খালি কান্দে
3,ছাত্রী টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছ...,ছাত্র টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছে...,[],0,0,0,0,<Gender> টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হ...,<Gender> টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হ...
4,"মেয়েরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ছে...","ছেলেরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর মে...",[],0,0,0,0,"<Gender> এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ...","<Gender> এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ..."
...,...,...,...,...,...,...,...,...,...
4854,অপুরভ ভাই অসাম,রাইসাভাবী অসাম,['অপুরভ '],1,2,2,0,অপুরভ <Gender> অসাম,<Name> <Gender> অসাম
4855,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা নূরজাহান আফা,['শিহাব'],1,1,1,0,সেই ছিল এটা শিহাব <Gender>,সেই ছিল এটা <Name> <Gender>
4856,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা সুস্মিতা বোন,['শিহাব'],1,1,1,0,সেই ছিল এটা শিহাব <Gender>,সেই ছিল এটা <Name> <Gender>
4857,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা গুলশান ভাবী,['শিহাব'],1,1,1,0,সেই ছিল এটা শিহাব <Gender>,সেই ছিল এটা <Name> <Gender>


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [18]:
temp = data[data['Predict Biasness']!= 1]
test = data[data['Predict Biasness']== 1]

In [19]:
t1 = temp[['Original Sentence', 'original_lebel']]
t2 = temp[['Converted Sentence', 'original_lebel']]
t2.columns = ['Original Sentence', 'original_lebel']
temp = pd.concat([t1, t2], axis = 0)
temp

,Original Sentence,original_lebel
0,সাইন্সের মেয়েরা আসলেই একটু অহংকারী ও দাম্বিক হ...,0
1,মেহজাবিন এর ছাত্রী এটা? এই ছাগীটাকে প্রায় নাটক...,0
2,মাইয়া টা খালি কান্দে,0
3,ছাত্রী টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছ...,0
4,"মেয়েরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ছে...",0
...,...,...
4854,রাইসাভাবী অসাম,1
4855,সেই ছিল এটা নূরজাহান আফা,1
4856,সেই ছিল এটা সুস্মিতা বোন,1
4857,সেই ছিল এটা গুলশান ভাবী,1


In [20]:
validation = temp.groupby('original_lebel', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

In [21]:
train

,Original Sentence,original_lebel
1,মেহজাবিন এর ছাত্রী এটা? এই ছাগীটাকে প্রায় নাটক...,0
5,ছাত্রী হিসেবে এই বান্দর ছ্যামড়ির ন্যাকামি দেইখ...,0
7,ওড়না ব্যবহার করলে কি খতি হত? খারাপ হয়ে যাচছে ব...,0
8,ওড়না ব্যবহার করলে কি খতি হত? খারাপ হয়ে যাচছে ব...,0
10,ছাত্র পড়ানো কঠিন প্যারা,0
...,...,...
4847,নাটকটা অনেক অনেক ভালো লাগলো । মুসফিকাভাবী কখনো...,1
4849,"কাহিনীচিত্র গুলো আমার সাথে মিলে গেলো, যদিও আমা...",1
4851,তাহমিদ ভাইয়ার অভিনয় খুব ভাল লেগেছে...?,1
4855,সেই ছিল এটা নূরজাহান আফা,1


In [23]:
validation

,Original Sentence,original_lebel
268,খুবই ফালতু নাটক। তাহসান আপার কাছে এটা আশা করিন...,0
91,ফালতু পরিচারিকা,0
726,ডাক্তার ও পাইছে একটা....!!! আবাল!!!,0
910,মীর ভাই অভিনয় মনে হয় ভূলে গেছে।,0
96,মোশারফকে আর এখন আগের মত ফেভারেট মনে হয় না উনি...,0
...,...,...
3020,মিসু ভাই দারুন,1
3702,শামীম ভাই যেখানে আছে নাটক সুপার??,1
3945,"সত্যিই অসাধারণ ছিলো.মিথিলার এমন করা উচিত হয়নি,...",1
2696,মিজানুর রহমান আরিয়ান ভাই মানেই অসাধারণ রোমান্টিক,1


In [24]:
test

,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked
11,"মেয়েরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...","ছেলেরা টেকায় না পড়লে কখনো মেয়েদের কাছে জায় না,...",[],0,1,0,1,<Gender> টেকায় না পড়লে কখনো <Gender> কাছে জায় ...,<Gender> টেকায় না পড়লে কখনো <Gender> কাছে জায় ...
13,পোলা খোর কখন আসবো,মাইয়া খোর কখন আসবো,[],0,0,1,1,<Gender> খোর কখন আসবো,<Gender> খোর কখন আসবো
14,পোলা খোর কখন আসবো,মেয়া খোর কখন আসবো,[],0,0,1,1,<Gender> খোর কখন আসবো,<Gender> খোর কখন আসবো
100,মোশাররফ ভাই আপনার প্রতিভা আছে কাজে লাগান এসব ফ...,মাইশা বোন আপনার প্রতিভা আছে কাজে লাগান এসব ফাল...,['মোশাররফ'],0,2,1,1,মোশাররফ <Gender> আপনার প্রতিভা আছে কাজে লাগান ...,<Name> <Gender> আপনার প্রতিভা আছে কাজে লাগান এ...
145,"শামীম ভাই আপনাকে দেখলে, আমার একটা নাটকের নাম ম...","কালীতারা বোন আপনাকে দেখলে, আমার একটা নাটকের না...",['শামীম'],0,2,0,1,"শামীম <Gender> আপনাকে দেখলে, আমার একটা নাটকের ...",<Name> <Gender> আপনাকে দেখলে আমার একটা নাটকের...
...,...,...,...,...,...,...,...,...,...
4827,নাটুক আসলে সেই লাগার মত অপূর্ব ভাইয়া আর তানজিন...,নাটুক আসলে সেই লাগার মত সাদিয়াআপু আর ইমদাদুল ভ...,"['অপূর্ব ','তানজিনা তিশা']",1,2,0,1,নাটুক আসলে সেই লাগার মত অপূর্ব <Gender> আর তান...,নাটুক আসলে সেই লাগার মত <Name> <Gender> আর <Na...
4833,নাটুক আসলে সেই লাগার মত অপূর্ব ভাইয়া আর তানজিন...,নাটুক আসলে সেই লাগার মত দুর্গাআপু আর শিব ভাইয়া,"['অপূর্ব ','তানজিনা তিশা']",1,2,0,1,নাটুক আসলে সেই লাগার মত অপূর্ব <Gender> আর তান...,নাটুক আসলে সেই লাগার মত <Name> <Gender> আর <Na...
4843,সত্যিই সিহাব ভাইজান তুমার প্রতি দোয়া রইরো আমাদ...,সত্যিই সিমন্তী বুবুজান তুমার প্রতি দোয়া রইরো আ...,['সিহাব'],1,1,0,1,সত্যিই সিহাব <Gender> তুমার প্রতি দোয়া রইরো আম...,সত্যিই <Name> <Gender> তুমার প্রতি দোয়া রইরো আ...
4852,অপুরভ ভাই অসাম,মুসফিকাআফা অসাম,['অপুরভ '],1,2,0,1,অপুরভ <Gender> অসাম,<Name> <Gender> অসাম


# Finetune with ner + original all data

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model

In [26]:
model_path = "ka05ar/banglabert-sentiment"
tokenizer, b_model = load_transformer_based_model(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at ka05ar/banglabert-sentiment and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

Total Pretrained tokens:  32000
Total number of tokens after adding the new ones:  32002


In [30]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column all data

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer, max_length=200 )
dataset_val= BertDataset( validation, tokenizer, max_length=200)
dataset_test= BertDataset(test, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle=False)


In [31]:
id,am, tid, l = next(iter(train_loader))
x = b_model(id, am,tid)
x.hidden_states[0].shape

torch.Size([16, 200, 768])

In [32]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [33]:
id,am, tid, l = next(iter(train_loader))
x,_ = masked_model(id, am,tid)
x.shape

torch.Size([16, 1])

In [34]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


In [37]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = '')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'/content/drive/MyDrive/#Research/# GB/saved_model/baseline_senti.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.028429,0.791737,0.024481,0.793103,113.449870,113.455558,0.0001,16,cuda
1,1,2,0.018665,0.856492,0.016288,0.867996,118.825154,235.664421,0.0001,16,cuda
2,1,3,0.010589,0.959696,0.011080,0.938578,118.522819,357.588582,0.0001,16,cuda
3,1,4,0.006332,0.980776,0.008194,0.966056,118.014678,479.900174,0.0001,16,cuda
4,1,5,0.004139,0.987690,0.006816,0.970905,118.170550,602.808162,0.0001,16,cuda
5,1,6,0.002878,0.993929,0.006069,0.971444,118.331604,725.545557,0.0001,16,cuda
6,1,7,0.002141,0.995447,0.005671,0.975216,118.264128,848.920261,0.0001,16,cuda
7,1,8,0.001720,0.995784,0.005385,0.975216,118.216918,972.224475,0.0001,16,cuda
8,1,9,0.001379,0.997470,0.005066,0.975216,118.202596,1095.328553,0.0001,16,cuda
9,1,10,0.001109,0.998651,0.004872,0.973599,118.108526,1218.221524,0.0001,16,cuda


Saved model and optimizer at /content/drive/MyDrive/#Research/# GB/saved_model/baseline_senti.pth


In [ ]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
  load_model(masked_model, None, PATH = '/content/drive/MyDrive/#Research/# GB/saved_model/baseline_senti.pth')

Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/Senti_Masked.pth


In [39]:

class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column all data

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()
dataset_test= BertDataset(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)

100%|██████████| 14/14 [00:02<00:00,  5.18it/s]


In [40]:
test['pred'] = [x[0] for pred in p for x in pred.tolist()]
test

<ipython-input-40-4aadd1bb9061>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = [x[0] for pred in p for x in pred.tolist()]


,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked,pred
11,"মেয়েরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...","ছেলেরা টেকায় না পড়লে কখনো মেয়েদের কাছে জায় না,...",[],0,1,0,1,<Gender> টেকায় না পড়লে কখনো <Gender> কাছে জায় ...,<Gender> টেকায় না পড়লে কখনো <Gender> কাছে জায় ...,0
13,পোলা খোর কখন আসবো,মাইয়া খোর কখন আসবো,[],0,0,1,1,<Gender> খোর কখন আসবো,<Gender> খোর কখন আসবো,0
14,পোলা খোর কখন আসবো,মেয়া খোর কখন আসবো,[],0,0,1,1,<Gender> খোর কখন আসবো,<Gender> খোর কখন আসবো,0
100,মোশাররফ ভাই আপনার প্রতিভা আছে কাজে লাগান এসব ফ...,মাইশা বোন আপনার প্রতিভা আছে কাজে লাগান এসব ফাল...,['মোশাররফ'],0,2,1,1,মোশাররফ <Gender> আপনার প্রতিভা আছে কাজে লাগান ...,<Name> <Gender> আপনার প্রতিভা আছে কাজে লাগান এ...,0
145,"শামীম ভাই আপনাকে দেখলে, আমার একটা নাটকের নাম ম...","কালীতারা বোন আপনাকে দেখলে, আমার একটা নাটকের না...",['শামীম'],0,2,0,1,"শামীম <Gender> আপনাকে দেখলে, আমার একটা নাটকের ...",<Name> <Gender> আপনাকে দেখলে আমার একটা নাটকের...,0
...,...,...,...,...,...,...,...,...,...,...
4827,নাটুক আসলে সেই লাগার মত অপূর্ব ভাইয়া আর তানজিন...,নাটুক আসলে সেই লাগার মত সাদিয়াআপু আর ইমদাদুল ভ...,"['অপূর্ব ','তানজিনা তিশা']",1,2,0,1,নাটুক আসলে সেই লাগার মত অপূর্ব <Gender> আর তান...,নাটুক আসলে সেই লাগার মত <Name> <Gender> আর <Na...,1
4833,নাটুক আসলে সেই লাগার মত অপূর্ব ভাইয়া আর তানজিন...,নাটুক আসলে সেই লাগার মত দুর্গাআপু আর শিব ভাইয়া,"['অপূর্ব ','তানজিনা তিশা']",1,2,0,1,নাটুক আসলে সেই লাগার মত অপূর্ব <Gender> আর তান...,নাটুক আসলে সেই লাগার মত <Name> <Gender> আর <Na...,1
4843,সত্যিই সিহাব ভাইজান তুমার প্রতি দোয়া রইরো আমাদ...,সত্যিই সিমন্তী বুবুজান তুমার প্রতি দোয়া রইরো আ...,['সিহাব'],1,1,0,1,সত্যিই সিহাব <Gender> তুমার প্রতি দোয়া রইরো আম...,সত্যিই <Name> <Gender> তুমার প্রতি দোয়া রইরো আ...,1
4852,অপুরভ ভাই অসাম,মুসফিকাআফা অসাম,['অপুরভ '],1,2,0,1,অপুরভ <Gender> অসাম,<Name> <Gender> অসাম,1


Result

In [41]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [42]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

218


# Prediction of NER Converted text data

In [44]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])
dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 14/14 [00:02<00:00,  4.86it/s]

Total test data:  218 Total mismatch:  36 Accuracy:  0.8348623853211009 Bias:  18



<ipython-input-44-8451d832cb32>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]


In [45]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  36 Accuracy:  0.8348623853211009 Bias:  18


In [46]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/baseline_senti_reSult.csv', index = False)